In [22]:
import pandas as pd
import json
from pydantic import BaseModel, TypeAdapter, RootModel
from PandasToPydantic.test.testData.libraryTypes import Book, Author, Library
from PandasToPydantic.test.config import LIBRARY_DATA_PATH
from PandasToPydantic.src.dataframeSerializer import expandAnnotation, getBaseFields, getListFields, serializeDataframe, dataframeToPydantic

In [23]:
data = pd.read_csv(LIBRARY_DATA_PATH)

In [24]:
libraryListRoot = dataframeToPydantic(data, Library)

In [25]:
libraryListRoot.root

[Library(LibraryID=1, LibraryName='City Central Library', Location='Cityville', EstablishedYear=1950, BookCollectionSize=50000, AuthorList=[Author(AuthorID=1, AuthorName='J.K. Rowling', AuthorBirthdate='1965-07-31', BookList=[Book(BookID=1, Title="Harry Potter and the Philosopher's Stone", Genre='Fantasy', PublishedYear=1997, AvailableCopies=5)]), Author(AuthorID=6, AuthorName='J.R.R. Tolkien', AuthorBirthdate='1892-01-03', BookList=[Book(BookID=11, Title='The Hobbit', Genre='Fantasy', PublishedYear=1937, AvailableCopies=2)]), Author(AuthorID=5, AuthorName='Mark Twain', AuthorBirthdate='1835-11-30', BookList=[Book(BookID=10, Title='The Adventures of Tom Sawyer', Genre='Adventure', PublishedYear=1876, AvailableCopies=2)])]),
 Library(LibraryID=2, LibraryName='Greenwood Public Library', Location='Greenwood', EstablishedYear=1975, BookCollectionSize=35000, AuthorList=[Author(AuthorID=2, AuthorName='George Orwell', AuthorBirthdate='1903-06-25', BookList=[Book(BookID=3, Title='1984', Genre=

In [32]:
libraryListRoot.model_dump()[3]["AuthorList"]

[{'AuthorID': 4,
  'AuthorName': 'Agatha Christie',
  'AuthorBirthdate': '1890-09-15',
  'BookList': [{'BookID': 7,
    'Title': 'Murder on the Orient Express',
    'Genre': 'Mystery',
    'PublishedYear': 1934,
    'AvailableCopies': 1}]},
 {'AuthorID': 5,
  'AuthorName': 'Mark Twain',
  'AuthorBirthdate': '1835-11-30',
  'BookList': [{'BookID': 9,
    'Title': 'Adventures of Huckleberry Finn',
    'Genre': 'Adventure',
    'PublishedYear': 1884,
    'AvailableCopies': 4},
   {'BookID': 10,
    'Title': 'The Adventures of Tom Sawyer',
    'Genre': 'Adventure',
    'PublishedYear': 1876,
    'AvailableCopies': 6}]},
 {'AuthorID': 2,
  'AuthorName': 'George Orwell',
  'AuthorBirthdate': '1903-06-25',
  'BookList': [{'BookID': 4,
    'Title': 'Animal Farm',
    'Genre': 'Political Satire',
    'PublishedYear': 1945,
    'AvailableCopies': 1}]},
 {'AuthorID': 6,
  'AuthorName': 'J.R.R. Tolkien',
  'AuthorBirthdate': '1892-01-03',
  'BookList': [{'BookID': 12,
    'Title': 'The Lord of the